In [ ]:
# !pip install sqlalchemy
# !pip install pyodbc

In [16]:
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

engine = create_engine('mssql://poseidon/sansys_readonly?trusted_connection=yes&driver=SQL+Server')



# ARRECADAÇÃO 
## Sem informação de estorno de pagamento

In [ ]:
%%time
data_pag_inicial = '20190101'
data_pag_final = '20191231 23:59:59.997'
a = engine.execute(f'''
set transaction isolation level read uncommitted 
select taa.nm_agente_arrecadador as "AGENTE ARRECADADOR"
    , ff.DT_EMISSAO_FATURA as "DATA EMISSÃO FATURA"
    , ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO"
    , cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA"
    , cc.NM_CLIENTE as "Morador"
    , cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável"
    , CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO
    , 'FATURA' as "TIPO DE DOCUMENTO"
    , ap.vl_valor_debito as "VALOR"
    , ff.vl_total_fatura as "VALOR DOCUMENTO"
    , CASE LEN(CAST(DATEPART( MM, ff.dt_mesano_ref) AS VARCHAR(2))) WHEN 1 THEN ('0' + CAST(DATEPART( MM, ff.dt_mesano_ref) AS VARCHAR(2))) ELSE CAST(DATEPART( MM, ff.dt_mesano_ref) AS VARCHAR(2)) END + '/' + CAST(DATEPART(YYYY, ff.dt_mesano_ref) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA"
    , ff.dt_vencimento_fatura as "VENCIMENTO"
    , tct.NM_CATEGORIA_TARIFA "CATEGORIA PRINCIPAL MATRÍCULA"
    , ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO"
from tab_agente_arrecadador taa
    inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
    inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
    inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
    inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
    inner join arr_pagamento_fatura apf on (ap.id_pagamento = apf.id_pagamento)
    inner join fat_fatura ff on (apf.id_fatura = ff.id_fatura)
    inner join cad_unidade_comercial cuc on (ff.ch_matricula_unidade = cuc.ch_matricula_unidade)
    left join TAB_CATEGORIA_TARIFA tct on (cuc.CH_CATEGORIA_TARIFA = tct.CH_CATEGORIA_TARIFA)
    inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
    inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
    left join FAT_FATURA_HISTORICO_CLIENTE ffhc on ffhc.ID_FATURA = ff.ID_FATURA and ffhc.CD_TIPO_CLIENTE_HISTORICO = 2
    left join CAD_CLIENTE cc on cc.CH_CLIENTE = ffhc.CH_CLIENTE
    left join (select ffs.ID_FATURA ,cast(fpsu.NR_PARCELA_SERVICO as varchar) + '/' + cast(fpsu.NR_TOTAL_PARCELA_SERVICO as varchar) as "Parcela"
    , fpsu.VL_PARCELA_SERVICO as "Valor"
        from FAT_FATURA_SERVICO ffs 
            join FAT_PARCELA_SERVICO_UNIDADE fpsu on fpsu.ID_PARCELA_SERVICO_UNIDADE = ffs.ID_PARCELA_SERVICO_UNIDADE
            join FAT_SERVICO_UNIDADE fsu on fsu.ID_SERVICO_UNIDADE = fpsu.ID_SERVICO_UNIDADE
            join TAB_SERVICO_DEFINICAO tsd on tsd.ID_SERVICO_DEFINICAO = fsu.ID_SERVICO_DEFINICAO and tsd.CD_SERVICO = 1060 )
    Parcelamento on Parcelamento.ID_FATURA = ff.ID_FATURA where ap.dt_pagamento_debito between '{data_pag_inicial}' and '{data_pag_final}' union
select taa.nm_agente_arrecadador as "AGENTE ARRECADADOR"
    , fad.DT_EMISSAO_AVISO_DEBITO as "DATA EMISSÃO FATURA"
    , ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO"
    , cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA"
    , null as "Morador"
    , total_fatura.NU_CPF_CNPJ as "CPF/CNPJ Responsável"
    , CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO
    , 'COMUNICADO' as "TIPO DE DOCUMENTO"
    , ap.vl_valor_debito as "VALOR"
    , fad.vl_total_aviso_debito as "VALOR DOCUMENTO"
    , CASE LEN(CAST(DATEPART( MM, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(2)))
    WHEN 1 THEN ('0' + CAST(DATEPART( MM, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(2))) 
    ELSE CAST(DATEPART( MM, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(2)) END + '/' + CAST(DATEPART(YYYY, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA"
    , fad.DT_VENCIMENTO_AVISO as "VENCIMENTO"
    , tct.NM_CATEGORIA_TARIFA "CATEGORIA PRINCIPAL MATRÍCULA"
    , ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO" 
from tab_agente_arrecadador taa inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador) 
    inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador) 
    inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban) 
    inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo) 
    inner join arr_pagamento_aviso_debito apad on (ap.id_pagamento = apad.id_pagamento) 
    inner join fat_aviso_debito fad on (apad.id_aviso_debito = fad.id_aviso_debito) 
    inner join cad_unidade_comercial cuc on (fad.ch_matricula_unidade = cuc.ch_matricula_unidade) 
    inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO) 
    inner join ( select SUM(vl_total_agua) vl_total_agua
        , SUM(vl_total_servicos) vl_total_servicos
        , SUM(vl_total_esgoto) vl_total_esgoto
        , faf.ID_AVISO_DEBITO, MAX(cc.NU_CPF_CNPJ) NU_CPF_CNPJ 
            from FAT_AVISO_DEBITO fad inner join FAT_AGRUPAMENTO_FATURA faf on (fad.ID_AVISO_DEBITO = faf.ID_AVISO_DEBITO) 
                inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA) 
                left join FAT_FATURA_HISTORICO_CLIENTE ffhc on ffhc.ID_FATURA = ff.ID_FATURA and ffhc.CD_TIPO_CLIENTE_HISTORICO = 2 
                left join CAD_CLIENTE cc on cc.CH_CLIENTE = ffhc.CH_CLIENTE group by faf.ID_AVISO_DEBITO ) 
    total_fatura on (fad.ID_AVISO_DEBITO = total_fatura.ID_AVISO_DEBITO) 
    left join TAB_CATEGORIA_TARIFA tct on (cuc.CH_CATEGORIA_TARIFA = tct.CH_CATEGORIA_TARIFA) 
    inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO) 
    where ap.dt_pagamento_debito between '{data_pag_inicial}' and '{data_pag_final}' union select taa.nm_agente_arrecadador as "AGENTE ARRECADADOR"
    , fsvf.DT_EMISSAO_SEGUNDA_VIA as "DATA EMISSÃO FATURA"
    , ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO"
    , cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA"
    , null as "Morador", total_fatura.NU_CPF_CNPJ as "CPF/CNPJ Responsável"
    , CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO
    , 'SEGUNDA VIA' as "TIPO DE DOCUMENTO"
    , ap.vl_valor_debito as "VALOR"
    , fsvf.vl_total_segunda_via as "VALOR DOCUMENTO"
    , CASE LEN(CAST(DATEPART( MM, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(2))) 
    WHEN 1 THEN ('0' + CAST(DATEPART( MM, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(2))) 
    ELSE CAST(DATEPART( MM, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(2)) END + '/' + CAST(DATEPART(YYYY, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA"
    , null as "VENCIMENTO"
    , tct.NM_CATEGORIA_TARIFA "CATEGORIA PRINCIPAL MATRÍCULA"
    , ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO" 
from tab_agente_arrecadador taa inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador) 
    inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador) 
    inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban) 
    inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo) 
    inner join arr_pagamento_segunda_via apsv on (ap.id_pagamento = apsv.id_pagamento) 
    inner join fat_segunda_via_fatura fsvf on (apsv.id_segunda_via_fatura = fsvf.id_segunda_via_fatura) 
    inner join cad_unidade_comercial cuc on (fsvf.ch_matricula_unidade = cuc.ch_matricula_unidade) 
    inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO) 
    inner join ( select SUM(vl_total_agua) vl_total_agua, SUM(vl_total_servicos) vl_total_servicos, SUM(vl_total_esgoto) vl_total_esgoto, faf.ID_SEGUNDA_VIA_FATURA, MAX(cc.NU_CPF_CNPJ) NU_CPF_CNPJ 
        from fat_segunda_via_fatura fsvf inner join FAT_AGRUPAMENTO_FATURA faf on (fsvf.ID_SEGUNDA_VIA_FATURA = faf.ID_SEGUNDA_VIA_FATURA) 
            inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA) 
            left join FAT_FATURA_HISTORICO_CLIENTE ffhc on ffhc.ID_FATURA = ff.ID_FATURA and ffhc.CD_TIPO_CLIENTE_HISTORICO = 2 
            left join CAD_CLIENTE cc on cc.CH_CLIENTE = ffhc.CH_CLIENTE group by faf.ID_SEGUNDA_VIA_FATURA ) 
    total_fatura on (fsvf.ID_SEGUNDA_VIA_FATURA = total_fatura.ID_SEGUNDA_VIA_FATURA) 
    left join TAB_CATEGORIA_TARIFA tct on (cuc.CH_CATEGORIA_TARIFA = tct.CH_CATEGORIA_TARIFA) 
    inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO) 
where ap.dt_pagamento_debito between '{data_pag_inicial}' and '{data_pag_final}' union 
    select taa.nm_agente_arrecadador as "AGENTE ARRECADADOR"
    , fcd.DT_EMISSAO_COBRANCA_DIVERSA as "DATA EMISSÃO FATURA"
    , ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO"
    , cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA", null as "Morador"
    , cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável"
    , CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO
    , 'COBRANÇA DIVERSA' as "TIPO DE DOCUMENTO"
    , ap.vl_valor_debito as "VALOR"
    , fcd.vl_total_cobranca_diversa as "VALOR DOCUMENTO"
    , CASE LEN(CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2))) 
    WHEN 1 THEN ('0' + CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2))) 
    ELSE CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)) END + '/' + CAST(DATEPART(YYYY, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA"
    , fcd.DT_VENCIMENTO_COBRANCA_DIVERSA as "VENCIMENTO"
    , tct.NM_CATEGORIA_TARIFA "CATEGORIA PRINCIPAL MATRÍCULA"
    , ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO" 
from tab_agente_arrecadador taa inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador) 
    inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador) 
    inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban) 
    inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo) 
    inner join arr_pagamento_cobranca_diversa apcd on (ap.id_pagamento = apcd.id_pagamento) 
    inner join fat_cobranca_diversa fcd on (apcd.id_cobranca_diversa = fcd.id_cobranca_diversa) 
    inner join cad_unidade_comercial cuc on (fcd.ch_matricula_unidade = cuc.ch_matricula_unidade) 
    inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO) 
    inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO) 
    left join CAD_CLIENTE cc on cc.CH_CLIENTE = fcd.CH_CLIENTE 
    left join TAB_CATEGORIA_TARIFA tct on (cuc.CH_CATEGORIA_TARIFA = tct.CH_CATEGORIA_TARIFA) 
where ap.dt_pagamento_debito between '{data_pag_inicial}' and '{data_pag_final}' union 
    select taa.nm_agente_arrecadador as "AGENTE ARRECADADOR"
    , fcd.DT_EMISSAO_COBRANCA_DIVERSA as "DATA EMISSÃO FATURA"
    , ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO"
    , cast(ccc.ch_matricula_cliente_comercial as varchar(10)) + '-' + ccc.digito_matricula_cliente_comercial as "MATRÍCULA"
    , null as "Morador", cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável"
    , '' GRUPO
    , 'COBRANÇA DIVERSA' as "TIPO DE DOCUMENTO"
    , ap.vl_valor_debito as "VALOR"
    , fcd.vl_total_cobranca_diversa as "VALOR DOCUMENTO"
    , CASE LEN(CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2))) 
WHEN 1 THEN ('0' + CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2))) 
    ELSE CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)) END + '/' + CAST(DATEPART(YYYY, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA"
    , fcd.DT_VENCIMENTO_COBRANCA_DIVERSA as "VENCIMENTO"
    , NULL CATEGORIA_PRINCIPAL, ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO" 
from tab_agente_arrecadador taa inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador) 
    inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador) 
    inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban) 
    inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo) 
    inner join arr_pagamento_cobranca_diversa apcd on (ap.id_pagamento = apcd.id_pagamento) 
    inner join fat_cobranca_diversa fcd on (apcd.id_cobranca_diversa = fcd.id_cobranca_diversa) 
    inner join cad_cliente_comercial ccc on (fcd.ch_matricula_cliente_comercial = ccc.ch_matricula_cliente_comercial) 
    inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO) 
    left join CAD_CLIENTE cc on cc.CH_CLIENTE = fcd.CH_CLIENTE 
where ap.dt_pagamento_debito between '{data_pag_inicial}' and '{data_pag_final}' union 
    select taa.nm_agente_arrecadador as "AGENTE ARRECADADOR"
    , fsfc.DT_EMISSAO_FATURA_CENTRALIZADA as "DATA EMISSÃO FATURA"
    , ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO"
    , cast(cuc.CD_ORGAO_CENTRALIZADOR as varchar(10)) as "MATRÍCULA", null as "Morador"
    , cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável"
    , CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO
    , 'FATURA CENTRALIZADA' as "TIPO DE DOCUMENTO"
    , ap.vl_valor_debito as "VALOR"
    , fsfc.VL_TOTAL_FATURA_CENTRALIZADA as "VALOR DOCUMENTO"
    , CASE LEN(CAST(DATEPART( MM, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(2))) 
    WHEN 1 THEN ('0' + CAST(DATEPART( MM, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(2))) 
    ELSE CAST(DATEPART( MM, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(2)) END + '/' + CAST(DATEPART(YYYY, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA"
    , fsfc.DT_VENCIMENTO_FATURA_CENTRALIZADA as "VENCIMENTO"
    , NULL CATEGORIA_PRINCIPAL
    , ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO" 
from tab_agente_arrecadador taa inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador) 
    inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador) 
    inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban) 
    inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo) 
    inner join ARR_PAGAMENTO_FATURA_CENTRALIZADA apfc on (ap.id_pagamento = apfc.id_pagamento) 
    inner join FAT_FATURA_CENTRALIZADA fsfc on (apfc.ID_FATURA_CENTRALIZADA = fsfc.ID_FATURA_CENTRALIZADA) 
    inner join CAD_ORGAO_CENTRALIZADOR cuc on (fsfc.ID_ORGAO_CENTRALIZADOR = cuc.ID_ORGAO_CENTRALIZADOR) 
    inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO) 
    inner join ( select SUM(vl_total_agua) vl_total_agua, SUM(vl_total_servicos) vl_total_servicos, SUM(vl_total_esgoto) vl_total_esgoto, faf.ID_FATURA_CENTRALIZADA 
from FAT_FATURA_CENTRALIZADA fsfc inner join FAT_AGRUPAMENTO_FATURA faf on (fsfc.ID_FATURA_CENTRALIZADA = faf.ID_FATURA_CENTRALIZADA) 
    inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA) group by faf.ID_FATURA_CENTRALIZADA ) total_fatura on (fsfc.ID_FATURA_CENTRALIZADA = total_fatura.ID_FATURA_CENTRALIZADA) 
    inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO) 
    left join CAD_CLIENTE cc on cc.CH_CLIENTE = cuc.CH_CLIENTE 
where ap.dt_pagamento_debito between '{data_pag_inicial}' and '{data_pag_final}' union select taa.nm_agente_arrecadador as "AGENTE ARRECADADOR"
    , null as "DATA EMISSÃO FATURA"
    , ap.DT_PAGAMENTO_DEBITO as "DATA PAGAMENTO"
    , '' as "MATRÍCULA"
    , null as "Morador"
    , '' as "CPF/CNPJ Responsável"
    , '' as GRUPO
    , 'NAO IDENTIFICADO' as "TIPO DE DOCUMENTO"
    , ap.vl_valor_debito as "VALOR"
    , ap.vl_valor_debito as "VALOR DOCUMENTO"
    , CASE LEN(CAST(DATEPART( MM, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(2))) 
    WHEN 1 THEN ('0' + CAST(DATEPART( MM, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(2))) 
    ELSE CAST(DATEPART( MM, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(2)) END + '/' + CAST(DATEPART(YYYY, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA"
    , ap.DT_VENCIMENTO_DEBITO as "VENCIMENTO"
    , NULL CATEGORIA_PRINCIPAL
    , ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO" 
from tab_agente_arrecadador taa inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador) 
    inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador) 
    inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban) 
    inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo) and (ap.ch_situacao_pagto = 2) 
    inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO) 
where ap.dt_pagamento_debito between '{data_pag_inicial}' and '{data_pag_final}' union select taa.NM_AGENTE_ARRECADADOR as "AGENTE ARRECADADOR"
    , null as "DATA EMISSÃO FATURA"
    , adb.DT_DEPOSITO as "DATA PAGAMENTO"
    , NULL as "MATRÍCULA"
    , null as "Morador"
    , '' as "CPF/CNPJ Responsável"
    , NULL as GRUPO
    , 'NAO IDENTIFICADO' as "TIPO DE DOCUMENTO"
    , adb.VL_DEPOSITADO as "VALOR"
    , NULL as "VALOR DOCUMENTO"
    , NULL as "REFERÊNCIA DA FATURA"
    , NULL as "VENCIMENTO"
    , NULL as "CATEGORIA_PRINCIPAL"
    , NULL as "FORMA PAGTO" 
from ARR_DEPOSITO_BANCARIO adb inner join TAB_AGENCIA ta on ta.ID_AGENCIA = adb.ID_AGENCIA 
    inner join TAB_AGENTE_ARRECADADOR taa on taa.ID_AGENTE_ARRECADADOR = ta.ID_AGENTE_ARRECADADOR 
    left join ARR_PAGAMENTO ap on ap.ID_DEPOSITO_BANCARIO = adb.ID_DEPOSITO_BANCARIO 
    where adb.FL_DEPOSITO_IDENTIFICADO = 'N' and adb.DT_DEPOSITO between '{data_pag_inicial}' and '{data_pag_final}' and not exists (select ID_DEPOSITO_BANCARIO from ARR_PAGAMENTO where ID_DEPOSITO_BANCARIO = adb.ID_DEPOSITO_BANCARIO)
''')
titulos = list(a.keys())
data = pd.DataFrame(a.fetchall(), columns=titulos)
data.to_csv('arr_2019.csv', sep=';', encoding='utf-8')

In [ ]:
data.to_csv('arr_2019.csv', sep=';', encoding='utf-8')

In [ ]:
# Função atualiza data
def FormarDataArr(dia_inicial, mes_inicial, ano_inicial, dia_final, mes_final, ano_final):
    while ano < ano_final:
        if mes < 10:
            data_inicial = str(ano) + '0' + str(mes) + '01'
            ultimo_dia = str((datetime(ano, mes, 1) + relativedelta(months=1, days=-1)).day)
            mes = mes + 1
        else:
            data_inicial = str(ano) + str(mes) + '0' + '01'
            if mes == 12:
                ano = ano + 1
                mes = 1
            else:
                mes = mes + 1
        print(data)
        RodarScriptFat(data)

In [ ]:
%%time
def RodarScriptArr(data_inicial, data_final):
    arr = pd.read_sql_query(f'''    
set transaction isolation level read uncommitted
SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                convert(varchar(10), aaf.DT_PROCESSAMENTO_SISTEMA, 103) as "DATA PROCESSAMENTO SISTEMA",
                ff.DT_EMISSAO_FATURA as "DATA EMISSÃO FATURA",
                convert(varchar(10), ap.DT_PAGAMENTO_DEBITO, 103) as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                aac.id_arquivo_conteudo as "ID ARQUIVO",
                ap.id_pagamento as "ID PAGAMENTO",
                cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'FATURA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                ff.vl_total_fatura as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, ff.dt_mesano_ref) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, ff.dt_mesano_ref) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, ff.dt_mesano_ref) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, ff.dt_mesano_ref) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                ff.dt_vencimento_fatura as "VENCIMENTO",
                ff.VL_TOTAL_AGUA "VALOR ÁGUA", ff.VL_TOTAL_ESGOTO "VALOR ESGOTO", ff.VL_TOTAL_SERVICOS "VALOR SERVIÇOS",
                tct.NM_CATEGORIA_TARIFA CATEGORIA_PRINCIPAL,
                ff.id_fatura as "ID DO DOCUMENTO",
                aac.de_conteudo_arquivo as "CONTEUDO ARQUIVO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                ap.VL_TARIFA as "TARIFA",
                cc.NM_CLIENTE as "Morador",
                Parcelamento.Parcela as "Parcela",
                Parcelamento.Valor as "Valor",
                cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                ff.vl_total_tbo_agua "TBO ÁGUA",
                ff.vl_total_tbo_esgoto "TBO ESGOTO"
from tab_agente_arrecadador taa
inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
inner join arr_pagamento_fatura apf on (ap.id_pagamento = apf.id_pagamento)
inner join fat_fatura ff on (apf.id_fatura = ff.id_fatura)
inner join cad_unidade_comercial cuc on (ff.ch_matricula_unidade = cuc.ch_matricula_unidade)
left join TAB_CATEGORIA_TARIFA tct on (ff.HIST_COD_CATEGORIA_PRINCIPAL = tct.CH_CATEGORIA_TARIFA)
inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
left join FAT_FATURA_HISTORICO_CLIENTE ffhc on ffhc.ID_FATURA = ff.ID_FATURA
                and ffhc.CD_TIPO_CLIENTE_HISTORICO = 2
left join CAD_CLIENTE cc on cc.CH_CLIENTE = ffhc.CH_CLIENTE

left join (select ffs.ID_FATURA
                               ,cast(fpsu.NR_PARCELA_SERVICO as varchar) + '/' + cast(fpsu.NR_TOTAL_PARCELA_SERVICO as varchar) as "Parcela"
                               ,fpsu.VL_PARCELA_SERVICO as "Valor"
                from FAT_FATURA_SERVICO ffs
                join FAT_PARCELA_SERVICO_UNIDADE fpsu on fpsu.ID_PARCELA_SERVICO_UNIDADE = ffs.ID_PARCELA_SERVICO_UNIDADE
                join FAT_SERVICO_UNIDADE fsu on fsu.ID_SERVICO_UNIDADE = fpsu.ID_SERVICO_UNIDADE

                join TAB_SERVICO_DEFINICAO tsd on tsd.ID_SERVICO_DEFINICAO = fsu.ID_SERVICO_DEFINICAO
                               and tsd.CD_SERVICO = 1060
) Parcelamento on Parcelamento.ID_FATURA = ff.ID_FATURA

where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                convert(varchar(10), aaf.DT_PROCESSAMENTO_SISTEMA, 103) as "DATA PROCESSAMENTO SISTEMA",
                fad.DT_EMISSAO_AVISO_DEBITO as "DATA EMISSÃO FATURA",
                convert(varchar(10), ap.DT_PAGAMENTO_DEBITO, 103) as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                aac.id_arquivo_conteudo as "ID ARQUIVO",
                ap.id_pagamento as "ID PAGAMENTO",
                cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'COMUNICADO' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fad.vl_total_aviso_debito as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fad.DT_EMISSAO_AVISO_DEBITO) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                fad.DT_VENCIMENTO_AVISO as "VENCIMENTO",
                total_fatura.vl_total_agua "VALOR ÁGUA", total_fatura.vl_total_esgoto "VALOR ESGOTO", total_fatura.vl_total_servicos "VALOR SERVIÇOS",
                categoria.todas CATEGORIA_PRINCIPAL,
                fad.id_aviso_debito as "ID DO DOCUMENTO",
                aac.de_conteudo_arquivo as "CONTEUDO ARQUIVO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                ap.VL_TARIFA as "TARIFA",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                total_fatura.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                total_fatura.vl_total_tbo_agua "TBO ÁGUA",
                total_fatura.vl_total_tbo_esgoto "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join arr_pagamento_aviso_debito apad on (ap.id_pagamento = apad.id_pagamento)
                inner join fat_aviso_debito fad on (apad.id_aviso_debito = fad.id_aviso_debito)
                inner join cad_unidade_comercial cuc on (fad.ch_matricula_unidade = cuc.ch_matricula_unidade)
                inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
                inner join
                               (
                                               select SUM(vl_total_agua) vl_total_agua, SUM(vl_total_servicos) vl_total_servicos,
                                                               SUM(vl_total_esgoto) vl_total_esgoto, faf.ID_AVISO_DEBITO, MAX(cc.NU_CPF_CNPJ) NU_CPF_CNPJ,
                                                               SUM(ISNULL(VL_TOTAL_TBO_AGUA,0)) vl_total_tbo_agua, SUM(ISNULL(vl_total_tbo_esgoto,0)) vl_total_tbo_esgoto
                                               from FAT_AVISO_DEBITO fad
                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fad.ID_AVISO_DEBITO = faf.ID_AVISO_DEBITO)
                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA)
                                                               left join FAT_FATURA_HISTORICO_CLIENTE ffhc on ffhc.ID_FATURA = ff.ID_FATURA and ffhc.CD_TIPO_CLIENTE_HISTORICO = 2
                                                               left join CAD_CLIENTE cc on cc.CH_CLIENTE = ffhc.CH_CLIENTE
                                               group  by faf.ID_AVISO_DEBITO
                               ) total_fatura on (fad.ID_AVISO_DEBITO = total_fatura.ID_AVISO_DEBITO)
                outer apply
                (
                               select STUFF
                               (
                                               (
                                                               select ', ' + tct.NM_CATEGORIA_TARIFA
                                                               from  FAT_AVISO_DEBITO fad
                                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fad.ID_AVISO_DEBITO = faf.ID_AVISO_DEBITO)
                                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA),
                                                                               TAB_CATEGORIA_TARIFA tct
                                                               where ff.HIST_COD_CATEGORIA_PRINCIPAL = tct.CH_CATEGORIA_TARIFA
                                                                               and total_fatura.ID_AVISO_DEBITO = fad.ID_AVISO_DEBITO
                                                               for xml path('')
                                               ), 1, 2, ''
                               ) as [todas]
                ) as categoria
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                convert(varchar(10), aaf.DT_PROCESSAMENTO_SISTEMA, 103) as "DATA PROCESSAMENTO SISTEMA",
                fsvf.DT_EMISSAO_SEGUNDA_VIA as "DATA EMISSÃO FATURA",
                convert(varchar(10), ap.DT_PAGAMENTO_DEBITO, 103) as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                aac.id_arquivo_conteudo as "ID ARQUIVO",
                ap.id_pagamento as "ID PAGAMENTO",
                cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'SEGUNDA VIA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fsvf.vl_total_segunda_via as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fsvf.DT_EMISSAO_SEGUNDA_VIA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                null as "VENCIMENTO",
                total_fatura.vl_total_agua "VALOR ÁGUA", total_fatura.vl_total_esgoto "VALOR ESGOTO", total_fatura.vl_total_servicos "VALOR SERVIÇOS",
                categoria.todas ATEGORIA_PRINCIPAL,
                fsvf.id_segunda_via_fatura as "ID DO DOCUMENTO",
                aac.de_conteudo_arquivo as "CONTEUDO ARQUIVO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                ap.VL_TARIFA as "TARIFA",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                total_fatura.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                total_fatura.vl_total_tbo_agua "TBO ÁGUA",
                total_fatura.vl_total_tbo_esgoto "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join arr_pagamento_segunda_via apsv on (ap.id_pagamento = apsv.id_pagamento)
                inner join fat_segunda_via_fatura fsvf on (apsv.id_segunda_via_fatura = fsvf.id_segunda_via_fatura)
                inner join cad_unidade_comercial cuc on (fsvf.ch_matricula_unidade = cuc.ch_matricula_unidade)
                inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
                inner join
                               (
                                               select SUM(vl_total_agua) vl_total_agua, SUM(vl_total_servicos) vl_total_servicos,
                                                               SUM(vl_total_esgoto) vl_total_esgoto, faf.ID_SEGUNDA_VIA_FATURA, MAX(cc.NU_CPF_CNPJ) NU_CPF_CNPJ,
                                                               SUM(ISNULL(VL_TOTAL_TBO_AGUA,0)) vl_total_tbo_agua, SUM(ISNULL(vl_total_tbo_esgoto,0)) vl_total_tbo_esgoto
                                               from fat_segunda_via_fatura fsvf
                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fsvf.ID_SEGUNDA_VIA_FATURA = faf.ID_SEGUNDA_VIA_FATURA)
                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA)
                                                               left join FAT_FATURA_HISTORICO_CLIENTE ffhc on ffhc.ID_FATURA = ff.ID_FATURA and ffhc.CD_TIPO_CLIENTE_HISTORICO = 2
                                                               left join CAD_CLIENTE cc on cc.CH_CLIENTE = ffhc.CH_CLIENTE
                                               group  by faf.ID_SEGUNDA_VIA_FATURA
                               ) total_fatura on (fsvf.ID_SEGUNDA_VIA_FATURA = total_fatura.ID_SEGUNDA_VIA_FATURA)
                outer apply
                (
                               select STUFF
                               (
                                               (
                                                               select ', ' + tct.NM_CATEGORIA_TARIFA
                                                               from  fat_segunda_via_fatura fsvf
                                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fsvf.ID_SEGUNDA_VIA_FATURA = faf.ID_SEGUNDA_VIA_FATURA)
                                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA),
                                                                               TAB_CATEGORIA_TARIFA tct
                                                               where ff.HIST_COD_CATEGORIA_PRINCIPAL = tct.CH_CATEGORIA_TARIFA
                                                                               and total_fatura.ID_SEGUNDA_VIA_FATURA = fsvf.ID_SEGUNDA_VIA_FATURA
                                                               for xml path('')
                                               ), 1, 2, ''
                               ) as [todas]
                ) as categoria

                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                convert(varchar(10), aaf.DT_PROCESSAMENTO_SISTEMA, 103) as "DATA PROCESSAMENTO SISTEMA",
                fcd.DT_EMISSAO_COBRANCA_DIVERSA as "DATA EMISSÃO FATURA",
                convert(varchar(10), ap.DT_PAGAMENTO_DEBITO, 103) as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                aac.id_arquivo_conteudo as "ID ARQUIVO",
                ap.id_pagamento as "ID PAGAMENTO",
                cast(cuc.ch_matricula_unidade as varchar(10)) + '-' + cuc.digito_matricula_unidade as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'COBRANÇA DIVERSA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fcd.vl_total_cobranca_diversa as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                fcd.DT_VENCIMENTO_COBRANCA_DIVERSA as "VENCIMENTO",
                0 "VALOR ÁGUA", 0 "VALOR ESGOTO", fcd.vl_total_cobranca_diversa "VALOR SERVIÇOS",NULL CATEGORIA_PRINCIPAL,
                fcd.id_cobranca_diversa as "ID DO DOCUMENTO",
                aac.de_conteudo_arquivo as "CONTEUDO ARQUIVO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                ap.VL_TARIFA as "TARIFA",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                NULL "TBO ÁGUA",
                NULL "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join arr_pagamento_cobranca_diversa apcd on (ap.id_pagamento = apcd.id_pagamento)
                inner join fat_cobranca_diversa fcd on (apcd.id_cobranca_diversa = fcd.id_cobranca_diversa)
                inner join cad_unidade_comercial cuc on (fcd.ch_matricula_unidade = cuc.ch_matricula_unidade)
                inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
                left join CAD_CLIENTE cc on cc.CH_CLIENTE = fcd.CH_CLIENTE
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                convert(varchar(10), aaf.DT_PROCESSAMENTO_SISTEMA, 103) as "DATA PROCESSAMENTO SISTEMA",
                fcd.DT_EMISSAO_COBRANCA_DIVERSA as "DATA EMISSÃO FATURA",
                convert(varchar(10), ap.DT_PAGAMENTO_DEBITO, 103) as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                aac.id_arquivo_conteudo as "ID ARQUIVO",
                ap.id_pagamento as "ID PAGAMENTO",
                cast(ccc.ch_matricula_cliente_comercial as varchar(10)) + '-' + ccc.digito_matricula_cliente_comercial as "MATRÍCULA",
                '' GRUPO,
                'COBRANÇA DIVERSA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fcd.vl_total_cobranca_diversa as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fcd.DT_EMISSAO_COBRANCA_DIVERSA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                fcd.DT_VENCIMENTO_COBRANCA_DIVERSA as "VENCIMENTO",
                0 "VALOR ÁGUA", 0 "VALOR ESGOTO", fcd.vl_total_cobranca_diversa "VALOR SERVIÇOS",NULL CATEGORIA_PRINCIPAL,
                fcd.id_cobranca_diversa as "ID DO DOCUMENTO",
                aac.de_conteudo_arquivo as "CONTEUDO ARQUIVO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                ap.VL_TARIFA as "TARIFA",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                NULL "TBO ÁGUA",
                NULL "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join arr_pagamento_cobranca_diversa apcd on (ap.id_pagamento = apcd.id_pagamento)
                inner join fat_cobranca_diversa fcd on (apcd.id_cobranca_diversa = fcd.id_cobranca_diversa)
                inner join cad_cliente_comercial ccc on (fcd.ch_matricula_cliente_comercial = ccc.ch_matricula_cliente_comercial)
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
                left join CAD_CLIENTE cc on cc.CH_CLIENTE = fcd.CH_CLIENTE
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                convert(varchar(10), aaf.DT_PROCESSAMENTO_SISTEMA, 103) as "DATA PROCESSAMENTO SISTEMA",
                fsfc.DT_EMISSAO_FATURA_CENTRALIZADA as "DATA EMISSÃO FATURA",
                convert(varchar(10), ap.DT_PAGAMENTO_DEBITO, 103) as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                aac.id_arquivo_conteudo as "ID ARQUIVO",
                ap.id_pagamento as "ID PAGAMENTO",
                cast(cuc.CD_ORGAO_CENTRALIZADOR as varchar(10)) as "MATRÍCULA",
                CAST(fgf.NU_GRUPO_FATURAMENTO AS VARCHAR(2)) GRUPO,
                'FATURA CENTRALIZADA' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                fsfc.VL_TOTAL_FATURA_CENTRALIZADA as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, fsfc.DT_REFER_FATURA_CENTRALIZADA) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                fsfc.DT_VENCIMENTO_FATURA_CENTRALIZADA as "VENCIMENTO",
                total_fatura.vl_total_agua "VALOR ÁGUA", total_fatura.vl_total_esgoto "VALOR ESGOTO", total_fatura.vl_total_servicos "VALOR SERVIÇOS",
                categoria.todas CATEGORIA_PRINCIPAL,
                fsfc.ID_FATURA_CENTRALIZADA as "ID DO DOCUMENTO",
                aac.de_conteudo_arquivo as "CONTEUDO ARQUIVO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                ap.VL_TARIFA as "TARIFA",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                cc.NU_CPF_CNPJ as "CPF/CNPJ Responsável",
                total_fatura.vl_total_tbo_agua "TBO ÁGUA",
                total_fatura.vl_total_tbo_esgoto "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo)
                inner join ARR_PAGAMENTO_FATURA_CENTRALIZADA apfc on (ap.id_pagamento = apfc.id_pagamento)
                inner join FAT_FATURA_CENTRALIZADA fsfc on (apfc.ID_FATURA_CENTRALIZADA = fsfc.ID_FATURA_CENTRALIZADA)
                inner join CAD_ORGAO_CENTRALIZADOR cuc on (fsfc.ID_ORGAO_CENTRALIZADOR = cuc.ID_ORGAO_CENTRALIZADOR)
                inner join FAT_GRUPO_FATURAMENTO fgf on (cuc.ID_GRUPO_FATURAMENTO = fgf.ID_GRUPO_FATURAMENTO)
                inner join
                               (
                                               select SUM(vl_total_agua) vl_total_agua, SUM(vl_total_servicos) vl_total_servicos,
                                                               SUM(vl_total_esgoto) vl_total_esgoto, faf.ID_FATURA_CENTRALIZADA,
                                                               SUM(ISNULL(VL_TOTAL_TBO_AGUA,0)) vl_total_tbo_agua, SUM(ISNULL(vl_total_tbo_esgoto,0)) vl_total_tbo_esgoto
                                               from FAT_FATURA_CENTRALIZADA fsfc
                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fsfc.ID_FATURA_CENTRALIZADA = faf.ID_FATURA_CENTRALIZADA)
                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA)
                                               group  by faf.ID_FATURA_CENTRALIZADA
                                ) total_fatura on (fsfc.ID_FATURA_CENTRALIZADA = total_fatura.ID_FATURA_CENTRALIZADA)
                outer apply
                (
                               select STUFF
                               (
                                               (
                                                               select ', ' + tct.NM_CATEGORIA_TARIFA
                                                               from  FAT_FATURA_CENTRALIZADA fsfc
                                                                               inner join FAT_AGRUPAMENTO_FATURA faf on (fsfc.ID_FATURA_CENTRALIZADA = faf.ID_FATURA_CENTRALIZADA)
                                                                               inner join FAT_FATURA ff on (faf.ID_FATURA = ff.ID_FATURA),
                                                                               TAB_CATEGORIA_TARIFA tct
                                                               where ff.HIST_COD_CATEGORIA_PRINCIPAL = tct.CH_CATEGORIA_TARIFA
                                                                               and total_fatura.ID_FATURA_CENTRALIZADA = fsfc.ID_FATURA_CENTRALIZADA
                                                               for xml path('')
                                               ), 1, 2, ''
                               ) as [todas]
                ) as categoria
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
                left join CAD_CLIENTE cc on cc.CH_CLIENTE = cuc.CH_CLIENTE
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

SELECT taa.nm_agente_arrecadador as "AGENTE ARRECADADOR",
                tcaa.cd_convenio as "CONVENIO",
                convert(varchar(15), aaf.NU_SEQUENCIAL_ARQUIVO) as "NSA",
                convert(varchar(10), aaf.DT_PROCESSAMENTO_SISTEMA, 103) as "DATA PROCESSAMENTO SISTEMA",
                null as "DATA EMISSÃO FATURA",
                convert(varchar(10), ap.DT_PAGAMENTO_DEBITO, 103) as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                aac.id_arquivo_conteudo as "ID ARQUIVO",
                ap.id_pagamento as "ID PAGAMENTO",
                '' as "MATRÍCULA",
                '' as GRUPO,
                'NAO IDENTIFICADO' as "TIPO DE DOCUMENTO",
                ap.vl_valor_debito as "VALOR",
                ap.vl_valor_debito as "VALOR DOCUMENTO",
                CASE LEN(CAST(DATEPART( MM, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(2)))
                WHEN 1 THEN ('0' + CAST(DATEPART( MM, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(2)))
                ELSE CAST(DATEPART( MM, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(2)) END + '/' +
                CAST(DATEPART(YYYY, ap.DT_REF_DOCUMENTO_ARRECADACAO) AS VARCHAR(4)) as "REFERÊNCIA DA FATURA",
                ap.DT_VENCIMENTO_DEBITO as "VENCIMENTO",
                NULL "VALOR ÁGUA", NULL "VALOR ESGOTO", NULL "VALOR SERVIÇOS",NULL CATEGORIA_PRINCIPAL,
                0 as "ID DO DOCUMENTO",
                aac.de_conteudo_arquivo as "CONTEUDO ARQUIVO",
                ttfa.NM_TIPO_FORMA_ARRECADACAO as "FORMA PAGTO",
                ap.VL_TARIFA as "TARIFA",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                '' as "CPF/CNPJ Responsável",
                NULL "TBO ÁGUA",
                NULL "TBO ESGOTO"
from tab_agente_arrecadador taa
                inner join tab_convenio_agente_arrecadador tcaa on (taa.id_agente_arrecadador = tcaa.id_agente_arrecadador)
                inner join arr_arquivo_febraban aaf on (tcaa.id_convenio_agente_arrecadador = aaf.id_convenio_agente_arrecadador)
                inner join arr_arquivo_conteudo aac on (aac.id_arquivo_febraban = aaf.id_arquivo_febraban)
                inner join arr_pagamento ap on (aac.id_arquivo_conteudo = ap.id_arquivo_conteudo) and (ap.ch_situacao_pagto = 2)
                inner join TAB_TIPO_FORMA_ARRECADACAO ttfa on (ap.CH_TIPO_FORMA_ARRECADACAO = ttfa.CH_TIPO_FORMA_ARRECADACAO)
where ap.dt_pagamento_debito between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/

union

select taa.NM_AGENTE_ARRECADADOR as "AGENTE ARRECADADOR",
                NULL as "CONVENIO",
                adb.NUMERO_DOCUMENTO_DEPOSITO as "NSA",
                convert(varchar(10), adb.DT_INCLUSAO, 103) as "DATA PROCESSAMENTO SISTEMA",
                null as "DATA EMISSÃO FATURA",
                convert(varchar(10), adb.DT_DEPOSITO, 103) as "DATA PAGAMENTO",
                ap.FL_ESTORNO_PAGAMENTO as "Estornado",
                NULL as "ID ARQUIVO",
                NULL as "ID PAGAMENTO",
                NULL as "MATRÍCULA",
                NULL as GRUPO,
                'NAO IDENTIFICADO' as "TIPO DE DOCUMENTO",
                adb.VL_DEPOSITADO as "VALOR",
                NULL as "VALOR DOCUMENTO",
                NULL as "REFERÊNCIA DA FATURA",
                NULL as "VENCIMENTO",
                NULL as "VALOR ÁGUA",
                NULL as "VALOR ESGOTO",
                NULL as "VALOR SERVIÇOS",
                NULL as "CATEGORIA_PRINCIPAL",
                NULL as "ID DO DOCUMENTO",
                NULL as "CONTEUDO ARQUIVO",
                NULL as "FORMA PAGTO",
                NULL as "TARIFA",
                null as "Morador",
                null as "Parcela",
                null as "Valor",
                '' as "CPF/CNPJ Responsável",
                NULL "TBO ÁGUA",
                NULL "TBO ESGOTO"
from ARR_DEPOSITO_BANCARIO adb
                inner join TAB_AGENCIA ta on ta.ID_AGENCIA = adb.ID_AGENCIA
                inner join TAB_AGENTE_ARRECADADOR taa on taa.ID_AGENTE_ARRECADADOR = ta.ID_AGENTE_ARRECADADOR
                left join ARR_PAGAMENTO ap on ap.ID_DEPOSITO_BANCARIO = adb.ID_DEPOSITO_BANCARIO
where adb.FL_DEPOSITO_IDENTIFICADO = 'N'
                and adb.DT_DEPOSITO between '{data_inicial}' and '{data_final} 23:59:59.997' /*informar o período desejado :: repetir estas mesmas data em todos os blocos do script*/
                and not exists (select ID_DEPOSITO_BANCARIO from ARR_PAGAMENTO where ID_DEPOSITO_BANCARIO = adb.ID_DEPOSITO_BANCARIO)
    ''', engine)
    arr.to_csv(data_inicial + 'fat.csv', sep=';', encoding='utf-8')
    print('Relatório ' + data_inicial + 'fat.csv salvo!')

# FATURAMENTO

In [2]:
from sqlalchemy import create_engine
import pyodbc
import pandas as pd

engine = create_engine('mssql://poseidon/sansys_readonly?trusted_connection=yes&driver=SQL+Server')

In [5]:
# Função atualiza data
def FormarDataFat(ano, mes, dia, ano_final):
    while ano < ano_final:
        if mes < 10:
            data = str(ano) + '0' + str(mes) + '0' + str(dia)
            mes = mes + 1
        else:
            data = str(ano) + str(mes) + '0' + str(dia)
            if mes == 12:
                ano = ano + 1
                mes = 1
            else:
                mes = mes + 1
        print(data)
        RodarScriptFat(data)
        

In [6]:
%%time
def RodarScriptFat(data):
    fat = pd.read_sql_query(f'''
    set transaction isolation level read uncommitted
    select cast(cuc.CH_MATRICULA_UNIDADE as varchar) + '-' + cuc.DIGITO_MATRICULA_UNIDADE as "Matrícula"
    ,cc.NM_CLIENTE as "Morador"
    ,right(convert(varchar(10), t.DT_TEMPO, 103), 7) as "Mês referência"
    ,fgf.NU_GRUPO_FATURAMENTO as "Grupo faturamento"
    ,tsf.NM_SITUACAO_FATURA as "Situação fatura"
    ,ca.SUM_QT_VOLUME_FATURADO as "Volume faturado água (m³)"
    ,ce.SUM_QT_VOLUME_FATURADO as "Volume faturado esgoto (m³)"
    ,ligacao_particular.QT_VOLUME_FATURADO as "Volume faturado (m³) - ligação particular"
    ,round(isnull(f.VL_AGUA_FATO,0),2) as "Faturamento água (R$)"
    ,round(isnull(f.VL_ESGOTO_FATO,0),2) as "Faturamento esgoto (R$)"
    ,round(isnull(f.VL_SERVICO_FATO,0),2) as "Faturamento serviços (R$)"
    ,round(isnull(f.VL_MULTAS_JUROS_FATO,0),2) as "Faturamento multas e juros (R$)"
    ,round(isnull(f.VL_CANCELAMENTO,0),2) as "Cancelamentos (R$)"
    ,round(isnull(f.VL_DESCONTO_VAZAMENTO_FATO,0),2) as "Abatimentos (R$)"
    ,Economias.SUM_NR_ECONOMIA as "Número economias"
    ,case when CicloLeitura1.CICLO_LEITURA is null then CicloLeitura2.CICLO_LEITURA else CicloLeitura1.CICLO_LEITURA end as "Ciclo leitura"
    ,tct.NM_CATEGORIA_TARIFA as "Categoria principal" 
    ,Faturamento.NM_RSF_TIPO_CONSUMO_DIM as "Tipo consumo faturado"
    ,tarifa.NM_TIPO_TARIFA "Tipo de Tarifa"
    ,tsla.NM_SITUACAO_LIGACAO_AGUA "Situação da LA"
    ,tsle.NM_SITUACAO_LIGACAO_ESGOTO "Situação da LE"
    ,ch.NU_HIDROMETRO "NR Hidrômetro"
    ,case when CicloLeitura1.ocorrencia is null then CicloLeitura2.ocorrencia else CicloLeitura1.ocorrencia end as "Ocorrência da Leitura"
    ,f.QT_FATURA "QT_FATURA (0/1)"
    ,format(ff.DT_MESANO_REF, 'MM/yyyy') "Referência Fatura"
    ,round(isnull(f.VL_TBO_AGUA_FATO,0),2) "TBO Água"
    ,round(isnull(f.VL_TBO_ESGOTO_FATO,0),2) "TBO Esgoto"
    from DWH_RSF_FATO_NOVO f
    join DWH_TEMPO_DIM t on t.ID_TEMPO_DIM = f.ID_TEMPO_DIM
    join FAT_FATURA ff on ff.ID_FATURA = f.ID_RSF_FATURA_FATO
    join TAB_SITUACAO_FATURA tsf on tsf.CH_SITUACAO_FATURA = ff.CH_SITUACAO_FATURA
    join CAD_UNIDADE_COMERCIAL cuc on cuc.CH_MATRICULA_UNIDADE = ff.CH_MATRICULA_UNIDADE
    left join CAD_CLIENTE_UNIDADE_COMERCIAL ccuc on (cuc.CH_MATRICULA_UNIDADE = ccuc.CH_MATRICULA_UNIDADE and ccuc.CH_ATIVO = 1 and ccuc.CH_TIPO_CLIENTE = 2 )
    left join CAD_CLIENTE cc on (ccuc.CH_CLIENTE = cc.CH_CLIENTE)
    left join TAB_CATEGORIA_TARIFA tct on tct.CH_CATEGORIA_TARIFA = cuc.CH_CATEGORIA_TARIFA
    join FAT_GRUPO_FATURAMENTO fgf on fgf.ID_GRUPO_FATURAMENTO = ff.ID_GRUPO_FATURAMENTO
    left join CAD_UNIDADE_LIGACAO_AGUA cula on (cuc.CH_MATRICULA_UNIDADE = cula.CH_MATRICULA_UNIDADE) and (cula.CH_ATIVO = 1)
    left join TAB_SITUACAO_LIGACAO_AGUA tsla on (cula.CH_SITUACAO_LIGACAO_AGUA = tsla.CH_SITUACAO_LIGACAO_AGUA)
    left join CAD_UNIDADE_LIGACAO_ESGOTO cule on (cuc.CH_MATRICULA_UNIDADE = cule.CH_MATRICULA_UNIDADE) and (cule.CH_ATIVO = 1)
    left join TAB_SITUACAO_LIGACAO_ESGOTO tsle on (cule.CH_SITUACAO_LIGACAO_ESGOTO = tsle.CH_SITUACAO_LIGACAO_ESGOTO)
    left join CAD_HIDROMETRO ch on (cula.ID_HIDROMETRO = ch.ID_HIDROMETRO)
    outer apply (
    select max(ttt.NM_TIPO_TARIFA) as NM_TIPO_TARIFA
    from CAD_ECONOMIA c
    join TAB_CATEGORIA_TIPO_TARIFA tctt on (c.ID_CATEGORIA_TIPO_TARIFA = tctt.ID_CATEGORIA_TIPO_TARIFA) 
    and (cuc.CH_TIPO_CATEGORIA_TARIFA = c.CH_TIPO_CATEGORIA_TARIFA) 
    join TAB_TIPO_TARIFA ttt on (tctt.CH_TIPO_TARIFA = ttt.CH_TIPO_TARIFA)
    where (cuc.CH_MATRICULA_UNIDADE = c.CH_MATRICULA_UNIDADE) and (c.CH_ATIVO = 1)
    ) tarifa
    outer apply (
    select fclp.QT_VOLUME_FATURADO
    from FAT_CONSUMO_LIGACAO_PARTICULAR fclp
    where cuc.CH_MATRICULA_UNIDADE = fclp.CH_MATRICULA_UNIDADE
    and ff.DT_MESANO_REF = fclp.DT_MES_REF_LEITURA
    ) ligacao_particular
    left join (select ca.ID_RSF_FATO_NOVO,
    sum(ca.QT_VOLUME_FATURADO) as "SUM_QT_VOLUME_FATURADO"
    from DWH_RSF_COMPONENTE_FATURA ca
    where ca.FL_ECONOMIA_CAGUA_FATO = 1
    group by ca.ID_RSF_FATO_NOVO
    ) ca on ca.ID_RSF_FATO_NOVO = f.ID_RSF_FATO_NOVO
    left join (select ce.ID_RSF_FATO_NOVO,
    sum(ce.QT_VOLUME_FATURADO) as "SUM_QT_VOLUME_FATURADO"
    from DWH_RSF_COMPONENTE_FATURA ce
    where ce.FL_ECONOMIA_CESGOTO_FATO = 1
    group by ce.ID_RSF_FATO_NOVO
    ) ce on ce.ID_RSF_FATO_NOVO = f.ID_RSF_FATO_NOVO
    outer apply (select max(drtcd.NM_RSF_TIPO_CONSUMO_DIM) NM_RSF_TIPO_CONSUMO_DIM
    from DWH_RSF_FATO_NOVO f2
    join DWH_RSF_HIDROMETRO_DIM h on h.CH_RSF_HIDROMETRO_DIM = f2.CH_RSF_HIDROMETRO_DIM
    left join DWH_RSF_TIPO_CONSUMO_DIM drtcd on drtcd.CH_RSF_TIPO_CONSUMO_DIM = f2.CH_RSF_TIPO_CONSUMO_DIM
    where f.ID_RSF_FATO_NOVO = f2.ID_RSF_FATO_NOVO
    and t.ID_TEMPO_DIM = f2.ID_TEMPO_DIM
    group by f2.ID_RSF_FATO_NOVO
    ,drtcd.NM_RSF_TIPO_CONSUMO_DIM
    ) Faturamento
    outer apply (select f2.ID_RSF_FATO_NOVO,
    sum(cf.NR_ECONOMIA) as "SUM_NR_ECONOMIA"
    from DWH_RSF_FATO_NOVO f2
    left join DWH_RSF_COMPONENTE_FATURA cf on cf.ID_RSF_FATO_NOVO = f2.ID_RSF_FATO_NOVO
    where f.ID_RSF_FATO_NOVO = f2.ID_RSF_FATO_NOVO
    and t.ID_TEMPO_DIM = f2.ID_TEMPO_DIM
    group by f2.ID_RSF_FATO_NOVO
    ) Economias
    outer apply (select fmlu.CH_MATRICULA_UNIDADE,
    dateDiff(day, fmlu2.RET_DT_LEITURA, fmlu.RET_DT_LEITURA) as "CICLO_LEITURA",
    fmlu.RET_CD_OCORRENCIA ocorrencia
    from FAT_MASSA_LEITURA_UNIDADE fmlu
    join FAT_OCORRENCIA_LEITURA fol on fmlu.RET_CD_OCORRENCIA = fol.CD_OCORRENCIA_LEITURA
    join (select fmlu2.CH_MATRICULA_UNIDADE,
    fmlu2.RET_DT_LEITURA
    from FAT_MASSA_LEITURA_UNIDADE fmlu2
    where fmlu2.DT_MES_REF_UNIDADE = dateAdd(month, -1, t.DT_TEMPO)
    ) fmlu2 on fmlu2.CH_MATRICULA_UNIDADE = fmlu.CH_MATRICULA_UNIDADE
    where fmlu.DT_MES_REF_UNIDADE = t.DT_TEMPO
    and fmlu.CH_MATRICULA_UNIDADE = cuc.CH_MATRICULA_UNIDADE
    ) CicloLeitura1
    outer apply (select cuc2.CH_MATRICULA_UNIDADE,
    dateDiff(day, flula2.DT_LEITURA_UNIDADE, flula.DT_LEITURA_UNIDADE) as "CICLO_LEITURA",
    flula.CD_OCORRENCIA_LIDA ocorrencia
    from FAT_LEITURA_UNIDADE_LIGACAO_AGUA flula
    join CAD_UNIDADE_LIGACAO_AGUA cula on cula.ID_UNIDADE_LIGACAO_AGUA = flula.ID_UNIDADE_LIGACAO_AGUA
    join CAD_UNIDADE_COMERCIAL cuc2 on cuc2.CH_MATRICULA_UNIDADE = cula.CH_MATRICULA_UNIDADE
    join (select cuc.CH_MATRICULA_UNIDADE,
    flula.DT_LEITURA_UNIDADE
    from FAT_LEITURA_UNIDADE_LIGACAO_AGUA flula
    join CAD_UNIDADE_LIGACAO_AGUA cula on cula.ID_UNIDADE_LIGACAO_AGUA = flula.ID_UNIDADE_LIGACAO_AGUA
    join CAD_UNIDADE_COMERCIAL cuc on cuc.CH_MATRICULA_UNIDADE = cula.CH_MATRICULA_UNIDADE
    where flula.DT_MES_REF_LEITURA = dateAdd(month, -1, t.DT_TEMPO)
    ) flula2 on flula2.CH_MATRICULA_UNIDADE = cuc2.CH_MATRICULA_UNIDADE
    where flula.DT_MES_REF_LEITURA = t.DT_TEMPO
    and cuc.CH_MATRICULA_UNIDADE = cuc2.CH_MATRICULA_UNIDADE
    ) CicloLeitura2
    where t.DT_TEMPO = '{data}'
    union all
    select 'Serviços, cobranças e VC' as "Matrícula"
    ,'' as "Morador"
    ,'' as "Mês referência"
    ,0 as "Grupo faturamento"
    ,'' as "Situação fatura"
    ,0 as "Volume faturado água (m³)"
    ,0 as "Volume faturado esgoto (m³)"
    ,0 as "Volume faturado (m³) - ligação particular"
    ,round(sum(isnull(f.VL_AGUA_FATO,0)),2) as "Faturamento água (R$)"
    ,round(sum(isnull(f.VL_ESGOTO_FATO,0)),2) as "Faturamento esgoto (R$)"
    ,round(sum(isnull(f.VL_SERVICO_FATO,0)),2) as "Faturamento serviços (R$)"
    ,round(sum(isnull(f.VL_MULTAS_JUROS_FATO,0)),2) as "Faturamento multas e juros (R$)"
    ,round(sum(isnull(f.VL_CANCELAMENTO,0)),2) as "Cancelamentos (R$)"
    ,round(sum(isnull(f.VL_DESCONTO_VAZAMENTO_FATO,0)),2) as "Abatimentos (R$)"
    ,0 as "Número economias"
    ,0 as "Ciclo leitura"
    ,'' as "Categoria principal" 
    ,'' as "Tipo consumo faturado"
    ,'' "Tipo de Tarifa"
    ,'' "Situação da LA"
    ,'' "Situação da LE"
    ,'' "NR Hidrômetro"
    ,0 as "Ocorrência da Leitura"
    ,0 "QT_FATURA (0/1)"
    ,'' "Referência Fatura"
    ,round(sum(isnull(f.VL_TBO_AGUA_FATO,0)),2) "TBO Água"
    ,round(sum(isnull(f.VL_TBO_ESGOTO_FATO,0)),2) "TBO Esgoto"
    from DWH_RSF_FATO_NOVO f
    join DWH_TEMPO_DIM t on t.ID_TEMPO_DIM = f.ID_TEMPO_DIM
    where t.DT_TEMPO = '{data}'
    and f.ID_RSF_FATURA_FATO is null
    order by 1
    ''', engine)
    fat.to_csv(data + 'fat.csv', sep=';', encoding='utf-8')
    print('Relatório ' + data + 'fat.csv salvo!')

Wall time: 0 ns


In [7]:
ano = 2020
mes = 1
dia = 1
ano_final = 2022

In [14]:
from datetime import datetime
from dateutil.relativedelta import relativedelta



last_date_of_month = datetime(ano, mes, 1) + relativedelta(months=1, days=-1)
last_date_of_month

datetime.datetime(2020, 1, 31, 0, 0)

In [24]:
ano = 2020
mes = 2
dia = 1
ano_final = 2022

ultimo_dia = str((datetime(ano, mes, 1) + relativedelta(months=1, days=-1)).day)
ultimo_dia

'29'

In [14]:
%%time
FormarData(ano, mes, dia, ano_final)

20200101
Relatório 20200101fat.csv salvo!
20200201
Relatório 20200201fat.csv salvo!
20200301
Relatório 20200301fat.csv salvo!
20200401
Relatório 20200401fat.csv salvo!
20200501
Relatório 20200501fat.csv salvo!
20200601
Relatório 20200601fat.csv salvo!
20200701
Relatório 20200701fat.csv salvo!
20200801
Relatório 20200801fat.csv salvo!
20200901
Relatório 20200901fat.csv salvo!
20201001
Relatório 20201001fat.csv salvo!
20201101
Relatório 20201101fat.csv salvo!
20201201
Relatório 20201201fat.csv salvo!
20210101
Relatório 20210101fat.csv salvo!
20210201
Relatório 20210201fat.csv salvo!
20210301
Relatório 20210301fat.csv salvo!
20210401
Relatório 20210401fat.csv salvo!
20210501
Relatório 20210501fat.csv salvo!
20210601
Relatório 20210601fat.csv salvo!
20210701
Relatório 20210701fat.csv salvo!
20210801
Relatório 20210801fat.csv salvo!
20210901
Relatório 20210901fat.csv salvo!
20211001
Relatório 20211001fat.csv salvo!
20211101
Relatório 20211101fat.csv salvo!
20211201
Relatório 20211201fat.csv